In [1]:
import os

import keras
import numpy as np

import tensorflow as tf
import tensorflow.lite as tflite

In [2]:
model = keras.models.load_model('./bees-wasps.h5')

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11215873 (42.79 MB)
Trainable params: 11215873 (42.79 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\surak\AppData\Local\Temp\tmpj_gvnvz_\assets


INFO:tensorflow:Assets written to: C:\Users\surak\AppData\Local\Temp\tmpj_gvnvz_\assets


In [5]:
with open('./bees-wasps.tflite', 'wb') as f:
    f.write(tflite_model)

In [6]:
file_size_bytes = os.stat('bees-wasps.tflite').st_size
file_size_mb = file_size_bytes / 1000000
print("Model file size:", file_size_mb, "MB")

Model file size: 44.8662 MB


In [7]:
interpreter = tf.lite.Interpreter('./bees-wasps.tflite')
interpreter.allocate_tensors()

In [8]:
interpreter.get_output_details()[0]['index']

13

In [9]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [10]:
test_url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
test_image = download_image(test_url)

In [11]:
target_size = (150, 150)
processed_image = prepare_image(test_image, target_size)

In [12]:
image_array = np.array(processed_image)
image_array = image_array.reshape((1,) + image_array.shape)
image_array = image_array / 255

In [15]:
r_value = image_array[0, 0, 0, 0]
r_value

0.9450980392156862

In [17]:
# Load TFLite model:
model_path = './bees-wasps.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# define input data (x is image prepared above)
input_data = image_array

# Set the input tensor with input data
interpreter.set_tensor(input_details[0]['index'], input_data)

# Invoke the interpreter to perform inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# The output_data now contains the model's prediction
print("Model Prediction:", output_data)

ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_conv2d_input:0 